## Load libraries

In [1]:
import os
import urllib
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
import math
from numpy.random import uniform, binomial, normal


import torch
import torch.utils.data
import torch.utils.data as data
from torch.optim.optimizer import Optimizer
from torch.autograd import Variable
from torch.nn.functional import softplus
from torch.distributions import kl_divergence
import torch.nn as nn
from torch.nn import MSELoss, CrossEntropyLoss
from torch.distributions import kl_divergence
import torch.backends.cudnn as cudnn
from torch.distributions.normal import Normal
import torch.utils.data
from torchvision.utils import make_grid, save_image
from torch.nn import Module
from torchvision import datasets, transforms






## Utils

In [2]:
def cprint(color, text, **kwargs):
    if color[0] == '*':
        pre_code = '1;'
        color = color[1:]
    else:
        pre_code = ''
    code = {
        'a': '30',
        'r': '31',
        'g': '32',
        'y': '33',
        'b': '34',
        'p': '35',
        'c': '36',
        'w': '37'
    }
    print("\x1b[%s%sm%s\x1b[0m" % (pre_code, code[color], text), **kwargs)
    sys.stdout.flush()

# (used in sub network)
def normal_parse_params(params, min_sigma=1e-3):
    """
    Take a Tensor (e. g. neural network output) and return
    torch.distributions.Normal distribution.
    This Normal distribution is component-wise independent,
    and its dimensionality depends on the input shape.
    First half of channels is mean of the distribution,
    the softplus of the second half is std (sigma), so there is
    no restrictions on the input tensor.
    min_sigma is the minimal value of sigma. I. e. if the above
    softplus is less than min_sigma, then sigma is clipped
    from below with value min_sigma. This regularization
    is required for the numerical stability and may be considered
    as a neural network architecture choice without any change
    to the probabilistic model.
    """
    n = params.shape[0]
    d = params.shape[1]
    mu = params[:, :d // 2]
    sigma_params = params[:, d // 2:]
    sigma = softplus(sigma_params)
    sigma = sigma.clamp(min=min_sigma)
    distr = Normal(mu, sigma)
    return distr

## (used in the next function)
def torch_onehot(y, Nclass):
    if y.is_cuda:
        y = y.type(torch.cuda.LongTensor)
    else:
        y = y.type(torch.LongTensor)
    y_onehot = torch.zeros((y.shape[0], Nclass)).type(y.type())
    # In your for loop
    y_onehot.scatter_(1, y.unsqueeze(1), 1)
    return y_onehot

## (used in the fit of the main network)
def gauss_cat_to_flat(x, input_dim_vec):
    output = []
    for idx, dim in enumerate(input_dim_vec):
        if dim == 1:
            output.append(x[:, idx].unsqueeze(1))
        elif dim > 1:
            oh_vec = torch_onehot(x[:, idx], dim).type(x.type())
            output.append(oh_vec)
        else:
            raise ValueError('Error, invalid dimension value')
    return torch.cat(output, dim=1)

## (also used in the fit of the main network)
def flat_to_gauss_cat(x, input_dim_vec):
    output = []
    cum_dims = 0
    for idx, dims in enumerate(input_dim_vec):
        if dims == 1:
            output.append(x[:, cum_dims].unsqueeze(1))
            cum_dims += 1

        elif dims > 1:
            output.append(x[:, cum_dims:cum_dims + dims].max(dim=1)[1].type(x.type()).unsqueeze(1))
            cum_dims += dims

        else:
            raise ValueError('Error, invalid dimension value')

    return torch.cat(output, dim=1)

def gauss_cat_to_flat_mask(x, input_dim_vec):
    output = []
    for idx, dim in enumerate(input_dim_vec):
        if dim == 1:
            output.append(x[:, idx].unsqueeze(1))
        elif dim > 1:
            oh_vec = x.new_ones(x.shape[0], dim) * x[:, idx].unsqueeze(1)
            output.append(oh_vec)
        else:
            raise ValueError('Error, invalid dimension value')
    return torch.cat(output, dim=1)

## (also used in the fit of the main network)
def to_variable(var=(), cuda=True, volatile=False):
    out = []
    for v in var:
        if isinstance(v, np.ndarray):
            v = torch.from_numpy(v).type(torch.FloatTensor)
        if not v.is_cuda and cuda:
            v = v.cuda()
        if not isinstance(v, Variable):
            v = Variable(v, volatile=volatile)
        out.append(v)
    return out


# Datafeed function which does something, seems to just be a class for the data?
class Datafeed(data.Dataset):

    def __init__(self, x_train, y_train=None, transform=None):
        self.data = x_train
        self.targets = y_train
        self.transform = transform

    def __getitem__(self, index):
        img = self.data[index]
        if self.transform is not None:
            img = self.transform(img)
        if self.targets is not None:
            return img, self.targets[index]
        else:
            return img

    def __len__(self):
        return len(self.data)

## Skip Connection

In [3]:

#"""
class SkipConnection(nn.Module):

    #Skip-connection over the sequence of layers in the constructor.
    #The module passes input data sequentially through these layers
    #and then adds original data to the result.

    def __init__(self, *args):
        super(SkipConnection, self).__init__()
        self.inner_net = nn.Sequential(*args)

    def forward(self, input):
        return input + self.inner_net(input)
#"""
def preact_leaky_MLPBlock(width):
    return SkipConnection(
        nn.LeakyReLU(),
        nn.BatchNorm1d(num_features=width),
        nn.Linear(width, width),
        )


## Fully connected neural networks

In [4]:
# FC Networks

#Non-leaky (not used)
"""
class MLP_prior_net(nn.Module):
    def __init__(self, input_dim, width, depth, latent_dim):
        super(MLP_prior_net, self).__init__()
        # input layer
        proposal_layers = [nn.Linear(input_dim*2, width), nn.ReLU(), nn.BatchNorm1d(num_features=width)]
        # body
        for i in range(depth-1):
            proposal_layers.append(MLPBlock(width))
        # output layer
        proposal_layers.append(
            nn.Linear(width, latent_dim * 2)
        )

        self.block = nn.Sequential(*proposal_layers)

    def forward(self, x):
        return self.block(x)


class MLP_recognition_net(nn.Module):
    def __init__(self, input_dim, width, depth, latent_dim):
        super(MLP_recognition_net, self).__init__()
        # input layer
        proposal_layers = [nn.Linear(input_dim, width), nn.ReLU(), nn.BatchNorm1d(num_features=width)]
        # body
        for i in range(depth-1):
            proposal_layers.append(MLPBlock(width))
        # output layer
        proposal_layers.append(
            nn.Linear(width, latent_dim * 2)
        )

        self.block = nn.Sequential(*proposal_layers)

    def forward(self, x):
        return self.block(x)


class MLP_generator_net(nn.Module):
    def __init__(self, input_dim, width, depth, latent_dim):
        super(MLP_generator_net, self).__init__()
        # input layer
        generative_layers = [nn.Linear(latent_dim, width), nn.LeakyReLU(), nn.BatchNorm1d(num_features=width)]
        # body
        for i in range(depth-1):
            generative_layers.append(
                # skip-connection from prior network to generative network
                leaky_MLPBlock(width))
        # output layer
        generative_layers.extend([
            nn.Linear(width,
                      input_dim),
        ])
        self.block = nn.Sequential(*generative_layers)

    def forward(self, x):
        return self.block(x)
"""
## Fully linear residual path preact models

#Prior net
class MLP_preact_prior_net(nn.Module):
    def __init__(self, input_dim, width, depth, latent_dim):
        super(MLP_preact_prior_net, self).__init__()
        # input layer
        proposal_layers = [nn.Linear(input_dim*2, width)]
        # body
        for i in range(depth-1):
            proposal_layers.append(preact_leaky_MLPBlock(width))
        # output layer
        proposal_layers.extend([nn.LeakyReLU(), nn.BatchNorm1d(num_features=width), nn.Linear(width, latent_dim * 2)])


        self.block = nn.Sequential(*proposal_layers)

    def forward(self, x):
        return self.block(x)

#Encoder
class MLP_preact_recognition_net(nn.Module):
    def __init__(self, input_dim, width, depth, latent_dim):
        super(MLP_preact_recognition_net, self).__init__()
        # input layer
        proposal_layers = [nn.Linear(input_dim, width)]
        # body
        for i in range(depth-1):
            proposal_layers.append(preact_leaky_MLPBlock(width))
        # output layer
        proposal_layers.extend([nn.LeakyReLU(), nn.BatchNorm1d(num_features=width), nn.Linear(width, latent_dim * 2)])

        self.block = nn.Sequential(*proposal_layers)

    def forward(self, x):
        return self.block(x)

#Decoder
class MLP_preact_generator_net(nn.Module):
    def __init__(self, input_dim, width, depth, latent_dim):
        super(MLP_preact_generator_net, self).__init__()
        # input layer
        generative_layers = [nn.Linear(latent_dim, width)]
        # body
        for i in range(depth-1):
            generative_layers.append(
                # skip-connection from prior network to generative network
                preact_leaky_MLPBlock(width))
        # output layer
        generative_layers.extend([
            nn.LeakyReLU(), nn.BatchNorm1d(num_features=width), nn.Linear(width, input_dim),
        ])
        self.block = nn.Sequential(*generative_layers)

    def forward(self, x):
        return self.block(x)

## Functions for the data

In [5]:

# Load the datasets in UCI
def load_UCI(dset_name, splits=10, seed=0, separate_targets=True, save_dir='data/'):
    mkdir(save_dir)

    if dset_name == 'wine':
        if not os.path.isfile(save_dir+'winequality-red.csv'):
            urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv",
                               filename=save_dir+'winequality-red.csv')
        data = pd.read_csv(save_dir+'winequality-red.csv', header=1, delimiter=';').values
        y_idx = [-1]

    elif dset_name == 'default_credit':
        if not os.path.isfile(save_dir + 'default of credit card clients.xls'):
            urllib.request.urlretrieve(
                "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls",
                filename=save_dir + 'default of credit card clients.xls')
        data = pd.read_excel(save_dir + 'default of credit card clients.xls', header=[0, 1], index_col=0, # delimiter="\s+"
                             ).values
        y_idx = [-1]  # OK


    else:
        raise Exception('Dataset name doesnt match any known datasets.')

    np.random.seed(seed)
    data = data[np.random.permutation(np.arange(len(data)))]

    kf = KFold(n_splits=splits)
    for j, (train_index, test_index) in enumerate(kf.split(data)):

        # Not sure what separate targets is
        if separate_targets:
            x_idx = list(range(data.shape[1]))
            for e in y_idx:
                x_idx.remove(x_idx[e])

            x_idx = np.array(x_idx)
            y_idx = np.array(y_idx)
            x_train, y_train = data[train_index, :], data[train_index, :]
            x_train, y_train = x_train[:, x_idx], y_train[:, y_idx]
            x_test, y_test = data[test_index, :], data[test_index, :]
            x_test, y_test = x_test[:, x_idx], y_test[:, y_idx]

            x_means, x_stds = x_train.mean(axis=0), x_train.std(axis=0)
            y_means, y_stds = y_train.mean(axis=0), y_train.std(axis=0)

            y_stds[y_stds < 1e-10] = 1
            x_stds[x_stds < 1e-10] = 1

            x_train = ((x_train - x_means) / x_stds).astype(np.float32)
            y_train = ((y_train - y_means) / y_stds).astype(np.float32)

            x_test = ((x_test - x_means) / x_stds).astype(np.float32)
            y_test = ((y_test - y_means) / y_stds).astype(np.float32)

            return x_train, x_test, x_means, x_stds, y_train, y_test, y_means, y_stds

        else:
            x_train, x_test = data[train_index, :], data[test_index, :]
            x_means, x_stds = x_train.mean(axis=0), x_train.std(axis=0)

            x_stds[x_stds < 1e-10] = 1

            x_train = ((x_train - x_means) / x_stds).astype(np.float32)
            x_test = ((x_test - x_means) / x_stds).astype(np.float32)

            return x_train, x_test, x_means, x_stds

def mkdir(paths):
    if not isinstance(paths, (list, tuple)):
        paths = [paths]
    for path in paths:
        if not os.path.isdir(path):
            os.makedirs(path, mode=0o777)

# Not sure why this is needed
def unnormalise_cat_vars(x, x_means, x_stds, input_dim_vec):
    input_dim_vec = np.array(input_dim_vec)
    unnorm_x = np.multiply(x, x_stds) + x_means

    fixed_unnorm = unnorm_x.round()
    fixed_unnorm -= fixed_unnorm.min(axis=0).reshape([1, fixed_unnorm.shape[1]])  # this sets all mins to 0
    for idx, dims in enumerate(input_dim_vec):
        if dims > 1:
            vec = fixed_unnorm[:, idx]
            vec[vec > dims - 1] = dims - 1
            fixed_unnorm[:, idx] = vec

    x[:, input_dim_vec > 1] = fixed_unnorm[:, input_dim_vec > 1]
    return x


In [6]:
names = ['wine', 'default_credit', 'compas', 'lsat']
widths = [350, 350, 350, 350] # Bigger than VAE because the task of modelling all conditionals is more complex
depths = [3, 3, 3, 3] # We go deeper because we are using residual models
latent_dims = [6, 8, 4, 4]
under_latent_dims = [6, 8, 4, 4] # following the original paper we set dim(u) = dim(z) with d>r [r is true manifold dim]
under_latent_dims2 = [4, 6, 3, 3] # following the original paper we set dim(u) = dim(z) with d>r [r is true manifold dim]

dname = 'default_credit'
print(dname)

x_train, x_test, x_means, x_stds = \
load_UCI(dset_name=dname, splits=10, seed=42, separate_targets=False, save_dir='../data/')

default_credit


In [7]:
from __future__ import division
import datetime
import os
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from random import seed
#from .UCI_loader import unnormalise_cat_vars


# TODO return mean and std for variables + train test split

"""
def check_data_file(fname):
    files = os.listdir(".")  # get the current directory listing
    print
    "Looking for file '%s' in the current directory..." % fname

    if fname not in files:
        print
        "'%s' not found! Downloading from GitHub..." % fname
        addr = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
        try:
            response = urllib2.urlopen(addr)
        except:
            response = urllib3.urlopen(addr)
        data = response.read()
        fileOut = open(fname, "w")
        fileOut.write(data)
        fileOut.close()
        print
        "'%s' download and saved locally.." % fname
    else:
        print
        "File found in current directory.."
"""

def check_data_file(fname):
    files = os.listdir(".")  # get the current directory listing
    print
    "Looking for file '%s' in the current directory..." % fname

    if fname not in files:
        print
        "'%s' not found! Downloading from GitHub..." % fname
        addr = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
        
        response = urllib.request.urlopen(addr)

        data = response.read()
        fileOut = open(fname, "wb")
        fileOut.write(data)
        fileOut.close()
        print
        "'%s' download and saved locally.." % fname
    else:
        print
        "File found in current directory.."

def get_my_COMPAS(rseed=0, separate_test=True, test_ratio=0.2, save_dir='../data/'):
    """
        The adult dataset can be obtained from: https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv
        The code will look for the data file in the present directory, if it is not found, it will download them from GitHub.
    """

    SEED = rseed
    seed(SEED)
    np.random.seed(SEED)

    their_FEATURES_CLASSIFICATION = ["age_cat", "race", "sex", "priors_count", "c_charge_degree"]
    FEATURES_CLASSIFICATION = ["age_cat", "race", "sex", "c_charge_degree", "is_recid", "priors_count",
                               "time_served"]  # features to be used for classification
    CONT_VARIABLES = ["priors_count",
                      "time_served"]  # continuous features, will need to be handled separately from categorical features, categorical features will be encoded using one-hot
    CLASS_FEATURE = "two_year_recid"  # the decision variable


    COMPAS_INPUT_FILE = save_dir + "compas-scores-two-years.csv"
    check_data_file(COMPAS_INPUT_FILE)

    # load the data and get some stats
    df = pd.read_csv(COMPAS_INPUT_FILE)
    df = df.dropna(subset=["days_b_screening_arrest"])  # dropping missing vals

    # convert to np array
    data = df.to_dict('list')
    for k in data.keys():
        data[k] = np.array(data[k])

    dates_in = data['c_jail_in']
    dates_out = data['c_jail_out']
    # this measures time in Jail
    time_served = []
    for i in range(len(dates_in)):
        di = datetime.datetime.strptime(dates_in[i], '%Y-%m-%d %H:%M:%S')
        do = datetime.datetime.strptime(dates_out[i], '%Y-%m-%d %H:%M:%S')
        time_served.append((do - di).days)
    time_served = np.array(time_served)
    time_served[time_served < 0] = 0
    data["time_served"] = time_served

    """ Filtering the data """

    # These filters are the same as propublica (refer to https://github.com/propublica/compas-analysis)
    # If the charge date of a defendants Compas scored crime was not within 30 days from when the person was arrested, we assume that because of data quality reasons, that we do not have the right offense.
    idx = np.logical_and(data["days_b_screening_arrest"] <= 30, data["days_b_screening_arrest"] >= -30)

    # We coded the recidivist flag -- is_recid -- to be -1 if we could not find a compas case at all.
    idx = np.logical_and(idx, data["is_recid"] != -1)

    # In a similar vein, ordinary traffic offenses -- those with a c_charge_degree of 'O' -- will not result in Jail time are removed (only two of them).
    idx = np.logical_and(idx, data["c_charge_degree"] != "O")  # F: felony, M: misconduct

    # We filtered the underlying data from Broward county to include only those rows representing people who had either recidivated in two years, or had at least two years outside of a correctional facility.
    idx = np.logical_and(idx, data["score_text"] != "NA")

    # select the examples that satisfy this criteria
    for k in data.keys():
        data[k] = data[k][idx]

    y = data[CLASS_FEATURE]
    """ Feature normalization and one hot encoding """

    print
    "\nNumber of people recidivating within two years"
    print
    pd.Series(y).value_counts()
    print
    "\n"

    X = []  # empty array with num rows same as num examples, will hstack the features to it
    X_dims = []

    feature_names = []
    for attr in FEATURES_CLASSIFICATION:
        vals = data[attr]
        if attr in CONT_VARIABLES:
            vals = [float(v) for v in vals]
            # vals = preprocessing.scale(vals, axis=0, with_mean=True, with_std=True)  # 0 mean and 1 variance
            vals = np.reshape(vals, (len(y), -1))  # convert from 1-d arr to a 2-d arr with one col
            X_dims.append(1)

        else:  # for binary categorical variables, the label binarizer uses just one var instead of two
            enc = preprocessing.OneHotEncoder(categories='auto', handle_unknown='error')
            enc.fit(vals.reshape(-1, 1))
            vals = enc.transform(vals.reshape(-1, 1)).todense()
            X_dims += [vals.shape[1]]*vals.shape[1]

        # add to learnable features
        X.append(vals)

        if attr in CONT_VARIABLES:  # continuous feature, just append the name
            feature_names.append(attr)
        else:  # categorical features
            if vals.shape[1] == 1:  # binary features that passed through lib binarizer
                feature_names.append(attr)
            else:
                for k in enc.categories_:  # non-binary categorical features, need to add the names for each cat
                    feature_names.append(attr + "_" + str(k))

    X = np.array(np.concatenate(list(X), axis=1))
    X_dims = np.array(X_dims)

    if separate_test:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=rseed, shuffle=True)

        x_means, x_stds = X_train.mean(axis=0), X_train.std(axis=0)
        x_means[X_dims>1] = 0
        x_stds[X_dims>1] = 1
        x_stds[x_stds < 1e-10] = 1

        x_train = ((X_train - x_means) / x_stds).astype(np.float32)
        x_test = ((X_test - x_means) / x_stds).astype(np.float32)

        return x_train, x_test, x_means, x_stds, y_train, y_test, feature_names, X_dims
    else:
        x_means, x_stds = X.mean(axis=0), X.std(axis=0)
        print(X_dims.shape, x_means.shape)
        x_means[:,X_dims>1] = 0
        x_stds[:,X_dims>1] = 1
        x_stds[x_stds < 1e-10] = 1

        x_train = ((X - x_means) / x_stds).astype(np.float32)

        return x_train, x_means, x_stds, y, feature_names, X_dims


def join_compas_targets(x_train, x_test, y_train, y_test, X_dims):
    # output from get method is onehot so we need to flatten and append 2
    input_dim_vec = X_dims_to_input_dim_vec(X_dims)
    input_dim_vec = np.append(input_dim_vec, 2)
    enc = preprocessing.OneHotEncoder(categories='auto', handle_unknown='error')
    enc.fit(y_train.reshape(-1, 1))

    vals_train = np.array(enc.transform(y_train.reshape(-1, 1)).todense()).astype(np.float32)
    vals_test = np.array(enc.transform(y_test.reshape(-1, 1)).todense()).astype(np.float32)

    x_train = np.concatenate([x_train, vals_train], axis=1)
    x_test = np.concatenate([x_test, vals_test], axis=1)
    return x_train, x_test, input_dim_vec


def X_dims_to_input_dim_vec(X_dims):
    """This is for our cat_Gauss VAE model"""
    input_dim_vec = []
    i = 0
    while i < len(X_dims):
        input_dim_vec.append(X_dims[i])
        i += X_dims[i]
    return np.array(input_dim_vec)

#"""
def input_dim_vec_to_X_dims(input_dim_vec):
    # This is for our cat_Gauss VAE model
    X_dims = []
    for i in input_dim_vec:
        for ii in range(i):
            X_dims.append(i)
    return np.array(X_dims)
#"""

# Create datafeed for torch

In [8]:
trainset = Datafeed(x_train, x_train, transform=None) 
valset = Datafeed(x_test, x_train, transform=None)

save_dir = '../saves/fc_preact_VAEAC_NEW_' + dname


# Masker for the VAEAC training

In [9]:
class top_masker:
    """
    Returned mask is sampled from component-wise independent Bernoulli
    distribution with probability of component to be unobserved p.
    Such mask induces the type of missingness which is called
    in literature "missing completely at random" (MCAR).
    If some value in batch is missed, it automatically becomes unobserved.
    """
    def __init__(self, p):
        self.p = p

    def __call__(self, batch):
        pp = uniform(low=0.0, high=self.p, size=batch.shape[0])
        pp = np.expand_dims(pp, axis=1)
        pp = np.repeat(pp, batch.shape[1], axis=1)
        nan_mask = torch.isnan(batch).float()  # missed values
#         bernoulli_mask_numpy = np.random.choice(2, size=batch.shape,
#                                                 p=[1 - pp, pp])
        bernoulli_mask_numpy = binomial(1, pp, size=None)
#         print(bernoulli_mask_numpy.shape)
        bernoulli_mask = torch.from_numpy(bernoulli_mask_numpy).float()
        mask = torch.max(bernoulli_mask, nan_mask)  # logical or
        return mask
    
masker = top_masker(p=1)



# rms cat loglike (only for categorical data)

In [10]:
class rms_cat_loglike(nn.Module):

    def __init__(self, input_dim_vec, reduction='none'):
        super(rms_cat_loglike, self).__init__()
        self.reduction = reduction
        self.input_dim_vec = input_dim_vec
        self.mse = MSELoss(reduction='none')  # takes(input, target)
        self.ce = CrossEntropyLoss(reduction='none')

    def forward(self, x, y):

        log_prob_vec = []
        cum_dims = 0
        for idx, dims in enumerate(self.input_dim_vec):
            if dims == 1:
                # Gaussian_case
                log_prob_vec.append(-self.mse(x[:, cum_dims], y[:, idx]).unsqueeze(1))
                cum_dims += 1

            elif dims > 1:
                if x.shape[1] == y.shape[1]:
                    raise Exception('Input and target seem to be in flat format. Need integer cat targets.')
                                
                if y.is_cuda:
                    tget = y[:, idx].type(torch.cuda.LongTensor)
                else:
                    tget = y[:, idx].type(torch.LongTensor)

                log_prob_vec.append(-self.ce(x[:, cum_dims:cum_dims + dims], tget).unsqueeze(1))
                cum_dims += dims

            else:
                raise ValueError('Error, invalid dimension value')

        log_prob_vec = torch.cat(log_prob_vec, dim=1)

        if self.reduction == 'none':
            return log_prob_vec
        elif self.reduction == 'sum':
            return log_prob_vec.sum()
        elif self.reduction == 'average':
            return log_prob_vec.mean()

# Gaussian log likelihood (For datasets with continous with or without categorical data)

In [11]:

class GaussianLoglike(Module):
    """
    Compute reconstruction log probability of groundtruth given
    a tensor of Gaussian distribution parameters and a mask.
    Gaussian distribution parameters are output of a neural network
    without any restrictions, the minimal sigma value is clipped
    from below to min_sigma (default: 1e-2) in order not to overfit
    network on some exact pixels.
    The first half of channels corresponds to mean, the second half
    corresponds to std. See normal_parse_parameters for more info.
    This layer doesn't work with NaNs in the data, it is used for
    inpainting. Roughly speaking, this loss is similar to L2 loss.
    Returns a vector of log probabilities for each object of the batch.
    """
    def __init__(self, min_sigma=1e-2):
        super(GaussianLoglike, self).__init__()
        self.min_sigma = min_sigma

    def forward(self, distr_params, groundtruth, mask=None):
        distr = normal_parse_params(distr_params, self.min_sigma)
        if mask is not None:
            log_probs = distr.log_prob(groundtruth) * mask
        else:
            log_probs = distr.log_prob(groundtruth)
        return log_probs.view(groundtruth.shape[0], -1).sum(-1)

# Rectified Adam Optimizer

In [12]:
class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))

        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt(
                            (1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (
                                        N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size * group['lr'], exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)

                p.data.copy_(p_data_fp32)

        return loss

# Base Network

In [13]:
class BaseNet(object):
    def __init__(self):
        cprint('c', '\nNet:')

    def get_nb_parameters(self):
        return np.sum(p.numel() for p in self.model.parameters())

    def set_mode_train(self, train=True):
        if train:
            self.model.train()
        else:
            self.model.eval()

    def update_lr(self, epoch, gamma=0.99):
        self.epoch += 1
        if self.schedule is not None:
            if len(self.schedule) == 0 or epoch in self.schedule:
                self.lr *= gamma
                print('learning rate: %f  (%d)\n' % (self.lr, epoch))
                for param_group in self.optimizer.param_groups:
                    param_group['lr'] = self.lr

    def save(self, filename):
        cprint('c', 'Writting %s\n' % filename)
        torch.save({
            'epoch': self.epoch,
            'lr': self.lr,
            'model': self.model,
            'optimizer': self.optimizer}, filename)

    def load(self, filename):
        cprint('c', 'Reading %s\n' % filename)
        state_dict = torch.load(filename)
        self.epoch = state_dict['epoch']
        self.lr = state_dict['lr']
        self.model = state_dict['model']
        self.optimizer = state_dict['optimizer']
        print('  restoring epoch: %d, lr: %f' % (self.epoch, self.lr))
        return self.epoch

# Sub network

In [14]:
class VAEAC_gauss(nn.Module):
    def __init__(self, input_dim, width, depth, latent_dim, pred_sig=True):
        super(VAEAC_gauss, self).__init__()
        self.latent_dim = latent_dim
        self.recognition_net = MLP_preact_recognition_net(input_dim, width, depth, latent_dim)
        self.prior_net = MLP_preact_prior_net(input_dim, width, depth, latent_dim)
        if pred_sig:
            self.generator_net = MLP_preact_generator_net(2*input_dim, width, depth, latent_dim)
            self.rec_loglike = GaussianLoglike(min_sigma=1e-2)
        else:
            self.generator_net = MLP_preact_generator_net(input_dim, width, depth, latent_dim)
            self.m_rec_loglike = MSELoss(reduction='none')
        self.pred_sig = pred_sig
        self.sigma_mu = 1e4
        self.sigma_sigma = 1e-4


    @staticmethod
    def apply_mask(x, mask):
        """Positive bits in mask are set to 0 in x (observed)"""
        observed = x.clone()  # torch.tensor(x)
        observed[mask.bool()] = 0
        return observed

    def recognition_encode(self, x):
        approx_post_params = self.recognition_net(x)
        approx_post = normal_parse_params(approx_post_params, 1e-3)
        return approx_post

    def prior_encode(self, x, mask):
        x = self.apply_mask(x, mask)
        x = torch.cat([x, mask], 1)
        prior_params = self.prior_net(x)
        prior = normal_parse_params(prior_params, 1e-3)
        return prior

    def decode(self, z_sample):
        rec_params = self.generator_net(z_sample)
        return rec_params

    def reg_cost(self, prior):
        num_objects = prior.mean.shape[0]
        mu = prior.mean.view(num_objects, -1)
        sigma = prior.scale.view(num_objects, -1)
        mu_regularizer = -(mu ** 2).sum(-1) / 2 / (self.sigma_mu ** 2)
        sigma_regularizer = (sigma.log() - sigma).sum(-1) * self.sigma_sigma
        return mu_regularizer + sigma_regularizer

    def vlb(self, prior, approx_post, x, rec_params):
        if self.pred_sig:
            rec = self.rec_loglike(rec_params, x).view(x.shape[0], -1).sum(-1)
        else:
            rec = -self.m_rec_loglike(rec_params, x).view(x.shape[0], -1).sum(-1)
        prior_regularization = self.reg_cost(prior).view(x.shape[0], -1).sum(-1)
        kl = kl_divergence(approx_post, prior).view(x.shape[0], -1).sum(-1)
        return rec - kl + prior_regularization

    def iwlb(self, prior, approx_post, x, K=50):
        estimates = []
        for i in range(K):
            latent = approx_post.rsample()
            rec_params = self.decode(latent)
            if self.pred_sig:
                rec_loglike = self.rec_loglike(rec_params, x).view(x.shape[0], -1).sum(-1)
            else:
                rec_loglike = -self.m_rec_loglike(rec_params, x).view(x.shape[0], -1).sum(-1)

            prior_log_prob = prior.log_prob(latent)
            prior_log_prob = prior_log_prob.view(x.shape[0], -1)
            prior_log_prob = prior_log_prob.sum(-1)

            proposal_log_prob = approx_post.log_prob(latent)
            proposal_log_prob = proposal_log_prob.view(x.shape[0], -1)
            proposal_log_prob = proposal_log_prob.sum(-1)

            estimate = rec_loglike + prior_log_prob - proposal_log_prob
            estimates.append(estimate[:, None])

        return torch.logsumexp(torch.cat(estimates, 1), 1) - np.log(K)

# Main Network

In [15]:
# Main Network
class VAEAC_gauss_net(BaseNet):
    def __init__(self, input_dim, width, depth, latent_dim, pred_sig=True, lr=1e-3, cuda=True):
        super(VAEAC_gauss_net, self).__init__()
        cprint('y', 'VAE_gauss_net')

        self.cuda = cuda

        self.input_dim = input_dim
        self.width = width
        self.depth = depth
        self.latent_dim = latent_dim
        self.lr = lr
        self.pred_sig = pred_sig

        self.create_net()
        self.create_opt()
        self.epoch = 0
        self.schedule = None

        self.vlb_scale = 1 / input_dim  # scale for dimensions of input so we can use same LR always

    def create_net(self):
        torch.manual_seed(42)
        torch.cuda.manual_seed(42)
        self.model = VAEAC_gauss(self.input_dim, self.width, self.depth, self.latent_dim, self.pred_sig)
        if self.cuda:
            self.model = self.model.cuda()
            cudnn.benchmark = True
        print('    Total params: %.2fM' % (self.get_nb_parameters() / 1000000.0))

    def create_opt(self):
        self.optimizer = RAdam(self.model.parameters(), lr=self.lr) # torch.optim.Adam

    def fit(self, x, mask):
        self.set_mode_train(train=True)

        x, mask = to_variable(var=(x, mask), cuda=self.cuda)
        self.optimizer.zero_grad()

        prior = self.model.prior_encode(x, mask)
        approx_post = self.model.recognition_encode(x)
        z_sample = approx_post.rsample()
        rec_params = self.model.decode(z_sample)

        vlb = self.model.vlb(prior, approx_post, x, rec_params)
        loss = (- vlb * self.vlb_scale).mean()

        loss.backward()
        self.optimizer.step()

        if self.pred_sig:
            rec_return = normal_parse_params(rec_params, 1e-3)
        else:
            rec_return = rec_params
        return vlb.mean().item(), rec_return

    def eval(self, x, mask, sample=False):
        self.set_mode_train(train=False)

        x, mask = to_variable(var=(x, mask), cuda=self.cuda)
        prior = self.model.prior_encode(x, mask)

        approx_post = self.model.recognition_encode(x)
        if sample:
            z_sample = approx_post.sample()
        else:
            z_sample = approx_post.loc
        rec_params = self.model.decode(z_sample)

        vlb = self.model.vlb(prior, approx_post, x, rec_params)

        if self.pred_sig:
            rec_return = normal_parse_params(rec_params, 1e-3)
        else:
            rec_return = rec_params
        return vlb.mean().item(), rec_return

    def eval_iw(self, x, mask, k=50):
        self.set_mode_train(train=False)
        x, mask = to_variable(var=(x, mask), cuda=self.cuda)

        prior = self.model.prior_encode(x, mask)
        approx_post = self.model.recognition_encode(x)

        iw_lb = self.model.iwlb(prior, approx_post, x, k)
        return iw_lb.mean().item()

    def get_prior(self, x, mask):
        self.set_mode_train(train=False)
        x, mask = to_variable(var=(x, mask), cuda=self.cuda)
        prior = self.model.prior_encode(x, mask)
        return prior

    def get_post(self, x):
        self.set_mode_train(train=False)
        x, = to_variable(var=(x,), cuda=self.cuda)
        approx_post = self.model.recognition_encode(x)
        return approx_post

    def inpaint(self, x, mask, Nsample=1, z_mean=False):
        self.set_mode_train(train=False)
        x, mask = to_variable(var=(x, mask), cuda=self.cuda)
        prior = self.model.prior_encode(x, mask)
        out = []
        for i in range(Nsample):
            if z_mean:
                z_sample = prior.loc.data
            else:
                z_sample = prior.sample()
            rec_params = self.model.decode(z_sample)
            out.append(rec_params.data)
        out = torch.stack(out, dim=0)

        if self.pred_sig:
            return [normal_parse_params(out[i], 1e-2) for i in range(Nsample)]
        else:
            return out

    def regenerate(self, z, grad=False):
        self.set_mode_train(train=False)
        if grad:
            if not z.requires_grad:
                z.requires_grad = True
        else:
            z, = to_variable(var=(z,), volatile=True, cuda=self.cuda)
        out = self.model.decode(z)
        if self.pred_sig:
            return normal_parse_params(out, 1e-2)
        else:
            return out.data
class VAEAC_gauss_cat(nn.Module):
    def __init__(self, input_dim_vec, width, depth, latent_dim, pred_sig=True):
        super(VAEAC_gauss_cat, self).__init__()

        self.input_dim = 0
        self.input_dim_vec = input_dim_vec
        for e in self.input_dim_vec:
            self.input_dim += e

        self.latent_dim = latent_dim
        self.recognition_net = MLP_preact_recognition_net(self.input_dim, width, depth, latent_dim)
        self.prior_net = MLP_preact_prior_net(self.input_dim, width, depth, latent_dim)
        if pred_sig:
            raise Exception('Not yet implemented')
        else:
            self.generator_net = MLP_preact_generator_net(self.input_dim, width, depth, latent_dim)
            self.rec_loglike = rms_cat_loglike(self.input_dim_vec, reduction='none')
        self.pred_sig = pred_sig
        self.sigma_mu = 1e4
        self.sigma_sigma = 1e-4


    @staticmethod
    def apply_mask(x, mask):
        """Positive bits in mask are set to 0 in x (observed)"""
        observed = x.clone()  # torch.tensor(x)
        observed[mask.bool()] = 0
        return observed

    def recognition_encode(self, x):
        """Works with flattened representATION"""
        approx_post_params = self.recognition_net(x)
        approx_post = normal_parse_params(approx_post_params, 1e-3)
        return approx_post

    def prior_encode(self, x, mask):
        """Works with flattened representATION"""
        x = self.apply_mask(x, mask)
        x = torch.cat([x, mask], 1)
        prior_params = self.prior_net(x)
        prior = normal_parse_params(prior_params, 1e-3)
        return prior

    def decode(self, z_sample):
        rec_params = self.generator_net(z_sample)
        return rec_params

    def reg_cost(self, prior):
        num_objects = prior.mean.shape[0]
        mu = prior.mean.view(num_objects, -1)
        sigma = prior.scale.view(num_objects, -1)
        mu_regularizer = -(mu ** 2).sum(-1) / 2 / (self.sigma_mu ** 2)
        sigma_regularizer = (sigma.log() - sigma).sum(-1) * self.sigma_sigma
        return mu_regularizer + sigma_regularizer

    def vlb(self, prior, approx_post, x, rec_params):
        if self.pred_sig:
            raise Exception('Not yet imeplemented')
        else:
            rec = self.rec_loglike(rec_params, x).view(x.shape[0], -1).sum(-1)
        prior_regularization = self.reg_cost(prior).view(x.shape[0], -1).sum(-1)
        kl = kl_divergence(approx_post, prior).view(x.shape[0], -1).sum(-1)
        return rec - kl + prior_regularization

    def iwlb(self, prior, approx_post, x, K=50):
        estimates = []
        for i in range(K):
            latent = approx_post.rsample()
            rec_params = self.decode(latent)
            if self.pred_sig:
                raise Exception('Not yet imeplemented')
            else:
                rec_loglike = self.rec_loglike(rec_params, x).view(x.shape[0], -1).sum(-1)

            prior_log_prob = prior.log_prob(latent)
            prior_log_prob = prior_log_prob.view(x.shape[0], -1)
            prior_log_prob = prior_log_prob.sum(-1)

            proposal_log_prob = approx_post.log_prob(latent)
            proposal_log_prob = proposal_log_prob.view(x.shape[0], -1)
            proposal_log_prob = proposal_log_prob.sum(-1)

            estimate = rec_loglike + prior_log_prob - proposal_log_prob
            estimates.append(estimate[:, None])

        return torch.logsumexp(torch.cat(estimates, 1), 1) - np.log(K)

In [16]:


class VAEAC_gauss_cat_net(BaseNet):
    def __init__(self, input_dim_vec, width, depth, latent_dim, pred_sig=False, lr=1e-3, cuda=True, flatten=True):
        super(VAEAC_gauss_cat_net, self).__init__()
        cprint('y', 'VAE_gauss_net')

        self.cuda = cuda

        self.input_dim = 0
        self.input_dim_vec = input_dim_vec
        for e in self.input_dim_vec:
            self.input_dim += e
        self.flatten = flatten
        if not self.flatten:
            pass
            # raise Exception('Error calculation not supported without flattening')

        self.width = width
        self.depth = depth
        self.latent_dim = latent_dim
        self.lr = lr
        self.pred_sig = pred_sig

        self.create_net()
        self.create_opt()
        self.epoch = 0
        self.schedule = None

        self.vlb_scale = 1 / len(self.input_dim_vec)  # scale for dimensions of input so we can use same LR always

    def create_net(self):
        torch.manual_seed(42)
        torch.cuda.manual_seed(42)
        self.model = VAEAC_gauss_cat(self.input_dim_vec, self.width, self.depth, self.latent_dim, self.pred_sig)
        if self.cuda:
            self.model = self.model.cuda()
            cudnn.benchmark = True
        print('    Total params: %.2fM' % (self.get_nb_parameters() / 1000000.0))

    def create_opt(self):
        self.optimizer = RAdam(self.model.parameters(), lr=self.lr)  # torch.optim.Adam

    def fit(self, x, mask):
        self.set_mode_train(train=True)
        
        #print("bef x: ", x.shape)
        if self.flatten:
            mask = gauss_cat_to_flat_mask(mask, self.input_dim_vec)
            x_flat = gauss_cat_to_flat(x, self.input_dim_vec)
        
        else:
            x_flat = x # X already flattened
            x = flat_to_gauss_cat(x, self.input_dim_vec) # unflattened x (used for loss computation)
        #print("aft x: ", x.shape)
        x_flat, x, mask = to_variable(var=(x_flat, x, mask), cuda=self.cuda)
        self.optimizer.zero_grad()

        prior = self.model.prior_encode(x_flat, mask)
        approx_post = self.model.recognition_encode(x_flat)
        z_sample = approx_post.rsample()
        rec_params = self.model.decode(z_sample)

        vlb = self.model.vlb(prior, approx_post, x, rec_params)
        loss = (- vlb * self.vlb_scale).mean()

        loss.backward()
        self.optimizer.step()

        if self.pred_sig:
            rec_return = normal_parse_params(rec_params, 1e-3)
        else:
            rec_return = rec_params
        return vlb.mean().item(), rec_return

    def eval(self, x, mask, sample=False):
        self.set_mode_train(train=False)

        if self.flatten:
            mask = gauss_cat_to_flat_mask(mask, self.input_dim_vec)
            x_flat = gauss_cat_to_flat(x, self.input_dim_vec)
        else:
            x_flat = x
            x = flat_to_gauss_cat(x, self.input_dim_vec)

        x_flat, x, mask = to_variable(var=(x_flat, x, mask), cuda=self.cuda)
        prior = self.model.prior_encode(x_flat, mask)

        approx_post = self.model.recognition_encode(x_flat)
        if sample:
            z_sample = approx_post.sample()
        else:
            z_sample = approx_post.loc
        rec_params = self.model.decode(z_sample)

        vlb = self.model.vlb(prior, approx_post, x, rec_params)

        if self.pred_sig:
            rec_return = normal_parse_params(rec_params, 1e-3)
        else:
            rec_return = rec_params
        return vlb.mean().item(), rec_return

    def eval_iw(self, x, mask, k=50):
        self.set_mode_train(train=False)

        if self.flatten:
            mask = gauss_cat_to_flat_mask(mask, self.input_dim_vec)
            x_flat = gauss_cat_to_flat(x, self.input_dim_vec)
        else:
            x_flat = x
            x = flat_to_gauss_cat(x, self.input_dim_vec)


        x_flat, x, mask = to_variable(var=(x_flat, x, mask), cuda=self.cuda)

        prior = self.model.prior_encode(x, mask)
        approx_post = self.model.recognition_encode(x_flat)

        iw_lb = self.model.iwlb(prior, approx_post, x, k)
        return iw_lb.mean().item()

    def get_prior(self, x, mask, flatten=None):
        self.set_mode_train(train=False)
        if flatten is None:
            flatten = self.flatten
        if flatten:
            mask = gauss_cat_to_flat_mask(mask, self.input_dim_vec)
            x = gauss_cat_to_flat(x, self.input_dim_vec)

        x, mask = to_variable(var=(x, mask), cuda=self.cuda)
        prior = self.model.prior_encode(x, mask)
        return prior

    def get_post(self, x, flatten=None):
        self.set_mode_train(train=False)
        if flatten is None:
            flatten = self.flatten
        if flatten:
            x = gauss_cat_to_flat(x, self.input_dim_vec)

        x, = to_variable(var=(x,), cuda=self.cuda)
        approx_post = self.model.recognition_encode(x)
        return approx_post

    def inpaint(self, x, mask, Nsample=1, z_mean=False, flatten=False, unflatten=False, cat_probs=False):
        self.set_mode_train(train=False)

        if flatten:
            mask = gauss_cat_to_flat_mask(mask, self.input_dim_vec)
            x = gauss_cat_to_flat(x, self.input_dim_vec)

        x, mask = to_variable(var=(x, mask), cuda=self.cuda)
        prior = self.model.prior_encode(x, mask)
        out = []
        for i in range(Nsample):
            if z_mean:
                z_sample = prior.loc.data
            else:
                z_sample = prior.sample()
            rec_params = self.model.decode(z_sample)
            out.append(rec_params.data)
        out = torch.stack(out, dim=0)

        if self.pred_sig:
            raise Exception('Not yet implemented')
        else:
            dim0 = out.shape[0]
            dim1 = out.shape[1]
            out = out.view(dim0*dim1, -1)
            if unflatten:
                out = flat_to_gauss_cat(out, self.input_dim_vec)
            else:
                out = selective_softmax(out, self.input_dim_vec, grad=False, cat_probs=cat_probs)
            out = out.view(dim0, dim1, -1)
            return out.data

    def regenerate(self, z, grad=False, unflatten=False):
        self.set_mode_train(train=False)
        if unflatten and grad:
            raise Exception('flatten and grad options are not compatible')
        if grad:
            if not z.requires_grad:
                z.requires_grad = True
        else:
            z, = to_variable(var=(z,), volatile=True, cuda=self.cuda)
        out = self.model.decode(z)
        if self.pred_sig:
            raise Exception('Not yet implemented')
        else:
            if unflatten:
                out = flat_to_gauss_cat(out, self.input_dim_vec)
            else:
                out = selective_softmax(out, self.input_dim_vec, grad=grad)
            return out.data

## Training function

In [17]:
def train_VAEAC(net, masker, name, batch_size, nb_epochs, trainset, valset, cuda,
                flat_ims=False, train_plot=False, Nclass=None, early_stop=None, script_mode=False):

    models_dir = name + '_models'
    results_dir = name + '_results'
    mkdir(models_dir)
    mkdir(results_dir)

    if cuda:
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True,
                                                  num_workers=0)
        valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=True,
                                                num_workers=0)

    else:
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=False,
                                                  num_workers=0)
        valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=False,
                                                num_workers=0)

## ---------------------------------------------------------------------------------------------------------------------
# net dims
    cprint('c', '\nNetwork:')

    epoch = 0

    ## ---------------------------------------------------------------------------------------------------------------------
    # train
    cprint('c', '\nTrain:')

    print('  init cost variables:')
    vlb_train = np.zeros(nb_epochs)
    vlb_dev = np.zeros(nb_epochs)
    iwlb_dev = np.zeros(nb_epochs)
    best_vlb = -np.inf
    best_epoch = 0

    nb_its_dev = 1

    tic0 = time.time()
    for i in range(epoch, nb_epochs):
        net.set_mode_train(True)

        tic = time.time()
        nb_samples = 0
        for x, y in trainloader:

            if flat_ims:
                x = x.view(x.shape[0], -1)
            if Nclass is not None:
                y_oh = torch_onehot(y, Nclass).type(x.type())
                x = torch.cat([x, y_oh], 1)

            mask = masker(x)
            cost, _ = net.fit(x, mask)

            vlb_train[i] += cost * len(x)
            nb_samples += len(x)

        vlb_train[i] /= nb_samples

        toc = time.time()

        # ---- print
        print("it %d/%d, vlb %f, " % (i, nb_epochs, vlb_train[i]), end="")
        cprint('r', '   time: %f seconds\n' % (toc - tic))
        net.update_lr(i)

        # ---- dev
        if i % nb_its_dev == 0:
            nb_samples = 0
            for j, (x, y) in enumerate(valloader):

                if flat_ims:
                    x = x.view(x.shape[0], -1)
                if Nclass is not None:
                    y_oh = torch_onehot(y, Nclass).type(x.type())
                    x = torch.cat([x, y_oh], 1)

                mask = masker(x)
                cost, rec_mean = net.eval(x, mask)
                # iwlb = net.eval_iw(x, mask, 25)

                vlb_dev[i] += cost * len(x)
                # iwlb_dev[i] += iwlb
                nb_samples += len(x)

            vlb_dev[i] /= nb_samples
            # iwlb_dev[i] /= nb_samples

            cprint('g', '    vlb %f (%f)\n' % (vlb_dev[i], best_vlb))

            if train_plot:
                xm = net.model.apply_mask(x, mask)
                
                xr = x.cpu()
                rec_inpaint = net.inpaint(xm, mask)
                try:
                    o = rec_mean.cpu()
                    rec_inpaint = rec_inpaint[0].cpu()
                except:
                    o = rec_mean.loc.cpu()
                    rec_inpaint = rec_inpaint[0].loc.cpu()

                if Nclass is not None:
                    xm = xm[:, :-Nclass]
                    rec_inpaint = rec_inpaint[:, :-Nclass]
                    xr = xr[:, :-Nclass]
                    o = o[:, :-Nclass]

                if len(x.shape) == 2:
                    side = int(np.sqrt(xm.shape[1]))
                    xm = xm.view(-1, 1, side, side).data
                    rec_inpaint = rec_inpaint.view(-1, 1, side, side).data
                    xr = xr.view(-1, 1, side, side).data
                    o = o.view(-1, 1, side, side).data

                import matplotlib.pyplot as plt
                plt.figure()
                dd = make_grid(torch.cat([xr[:10], o[:10]]), nrow=10).numpy()
                plt.imshow(np.transpose(dd, (1, 2, 0)), interpolation='nearest')
                plt.title('reconstruct')
                if script_mode:
                    plt.savefig(results_dir + '/rec%d.png' % i)
                else:
                    plt.show()

                import matplotlib.pyplot as plt
                plt.figure()
                dd = make_grid(torch.cat([xm[:10], rec_inpaint[:10]]), nrow=10).numpy()
                plt.imshow(np.transpose(dd, (1, 2, 0)), interpolation='nearest')
                plt.title('inpaint')
                if script_mode:
                    plt.savefig(results_dir + '/inp%d.png' % i)
                else:
                    plt.show()

        if vlb_dev[i] > best_vlb:
            best_vlb = vlb_dev[i]
            best_epoch = i
            net.save(models_dir + '/theta_best.dat')

        if early_stop is not None and (i - best_epoch) > early_stop:
            break


    net.save(models_dir + '/theta_last.dat')
    toc0 = time.time()
    runtime_per_it = (toc0 - tic0) / float(nb_epochs)
    cprint('r', '   average time: %f seconds\n' % runtime_per_it)

    ## ---------------------------------------------------------------------------------------------------------------------
    # results
    cprint('c', '\nRESULTS:')
    nb_parameters = net.get_nb_parameters()
    best_cost_dev = np.max(vlb_dev)
    # best_iw_dev = np.max(iwlb_dev)
    best_cost_train = np.max(vlb_train)

    print('  best_vlb_dev: %f' % best_cost_dev)
    # print('  best_iwlb_dev: %f' % best_iw_dev)
    print('  best_vlb_train: %f' % best_cost_train)
    print('  nb_parameters: %d (%s)\n' % (nb_parameters, humansize(nb_parameters)))

    ## ---------------------------------------------------------------------------------------------------------------------
    # fig cost vs its
    import matplotlib.pyplot as plt

    plt.figure()
    plt.plot(np.clip(vlb_train, -1000, 1000), 'r')
    plt.plot(np.clip(vlb_dev[::nb_its_dev], -1000, 1000), 'b')
    plt.legend(['cost_train', 'cost_dev'])
    plt.ylabel('vlb')
    plt.xlabel('it')
    plt.grid(True)
    plt.savefig(results_dir+'/train_cost.png')

    # plt.figure()
    # plt.plot(np.clip(iwlb_dev[::nb_its_dev], -1000, 1000), 'b')
    # plt.ylabel('dev iwlb')
    # plt.xlabel('it')
    # plt.grid(True)
    # plt.savefig(results_dir + '/train_iwlb.png')
    if train_plot:
        plt.show()
    return vlb_train, vlb_dev


## COMPAS

In [20]:
masker = top_masker(p=1)

x_train, x_test, x_means, x_stds, y_train, y_test, feature_names, X_dims = \
    get_my_COMPAS(rseed=42, separate_test=True, test_ratio=0.1, save_dir='../data/')
x_train, x_test, input_dim_vec = join_compas_targets(x_train, x_test, y_train, y_test, X_dims)
print('Compas', x_train.shape, x_test.shape)
print(input_dim_vec)

dname = 'compas'
print(dname)

trainset = Datafeed(x_train, x_train, transform=None)
valset = Datafeed(x_test, x_test, transform=None)

save_dir = '../saves/fc_preact_VAEAC_NEW_' + dname

width = widths[names.index(dname)]
depth = depths[names.index(dname)] # number of hidden layers
latent_dim = latent_dims[names.index(dname)]

batch_size = 128
nb_epochs = 2000
early_stop = 200
lr = 1e-4

cuda = torch.cuda.is_available()

net = VAEAC_gauss_cat_net(input_dim_vec, width, depth, latent_dim, pred_sig=False, lr=lr,
                          cuda=cuda, flatten=False)

vlb_train, vlb_dev = train_VAEAC(net, masker, save_dir, batch_size, nb_epochs, trainset, valset, cuda,
                     flat_ims=False, train_plot=False, Nclass=None, early_stop=early_stop)

Compas (5554, 19) (618, 19)
[3 6 2 2 2 1 1 2]
compas

Net:
VAE_gauss_net
    Total params: 0.78M

Network:

Train:
  init cost variables:


/var/folders/38/m6rs_5bd7zsf3gjy2hg8y57r0000gn/T/ipykernel_31096/1070947545.py:6: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(p.numel() for p in self.model.parameters())


it 0/2000, vlb -18.383537,    time: 1.619305 seconds

    vlb -10.621977 (-inf)

Writting ../saves/fc_preact_VAEAC_NEW_compas_models/theta_best.dat

it 1/2000, vlb -12.485022,    time: 1.502898 seconds

    vlb -11.428103 (-10.621977)

it 2/2000, vlb -11.150703,    time: 1.491699 seconds

    vlb -10.307267 (-10.621977)

Writting ../saves/fc_preact_VAEAC_NEW_compas_models/theta_best.dat

it 3/2000, vlb -10.111939,    time: 1.530016 seconds

    vlb -9.356564 (-10.307267)

Writting ../saves/fc_preact_VAEAC_NEW_compas_models/theta_best.dat

it 4/2000, vlb -9.358087,    time: 1.516967 seconds

    vlb -8.211029 (-9.356564)

Writting ../saves/fc_preact_VAEAC_NEW_compas_models/theta_best.dat

it 5/2000, vlb -8.774173,    time: 1.523422 seconds

    vlb -7.624780 (-8.211029)

Writting ../saves/fc_preact_VAEAC_NEW_compas_models/theta_best.dat

it 6/2000, vlb -8.383996,    time: 1.521481 seconds

    vlb -7.104462 (-7.624780)

Writting ../saves/fc_preact_VAEAC_NEW_compas_models/theta_best.dat


it 51/2000, vlb -4.049516,    time: 1.517944 seconds

    vlb -3.030351 (-2.956059)

it 52/2000, vlb -4.046601,    time: 1.590051 seconds

    vlb -3.142626 (-2.956059)

it 53/2000, vlb -3.993357,    time: 1.557519 seconds

    vlb -3.010762 (-2.956059)

it 54/2000, vlb -4.010660,    time: 2.364297 seconds

    vlb -2.956914 (-2.956059)

it 55/2000, vlb -3.989221,    time: 2.354759 seconds

    vlb -2.935410 (-2.956059)

Writting ../saves/fc_preact_VAEAC_NEW_compas_models/theta_best.dat

it 56/2000, vlb -3.974315,    time: 2.010612 seconds

    vlb -2.773743 (-2.935410)

Writting ../saves/fc_preact_VAEAC_NEW_compas_models/theta_best.dat

it 57/2000, vlb -3.950603,    time: 1.889439 seconds

    vlb -3.053717 (-2.773743)

it 58/2000, vlb -3.914890,    time: 2.112087 seconds

    vlb -3.303768 (-2.773743)

it 59/2000, vlb -3.884073,    time: 1.887808 seconds

    vlb -3.060243 (-2.773743)

it 60/2000, vlb -3.932626,    time: 1.934271 seconds

    vlb -2.916941 (-2.773743)

it 61/2000, vl

    vlb -2.516448 (-2.463398)

it 122/2000, vlb -3.386675,    time: 2.037955 seconds

    vlb -2.574922 (-2.463398)

it 123/2000, vlb -3.378626,    time: 1.908550 seconds

    vlb -2.528915 (-2.463398)

it 124/2000, vlb -3.333230,    time: 1.910460 seconds

    vlb -2.438495 (-2.463398)

Writting ../saves/fc_preact_VAEAC_NEW_compas_models/theta_best.dat

it 125/2000, vlb -3.356431,    time: 1.969429 seconds

    vlb -2.497544 (-2.438495)

it 126/2000, vlb -3.335909,    time: 1.925340 seconds

    vlb -2.429570 (-2.438495)

Writting ../saves/fc_preact_VAEAC_NEW_compas_models/theta_best.dat

it 127/2000, vlb -3.351274,    time: 1.817053 seconds

    vlb -2.466796 (-2.429570)

it 128/2000, vlb -3.335552,    time: 1.733747 seconds

    vlb -2.373957 (-2.429570)

Writting ../saves/fc_preact_VAEAC_NEW_compas_models/theta_best.dat

it 129/2000, vlb -3.395688,    time: 1.792413 seconds

    vlb -2.484561 (-2.373957)

it 130/2000, vlb -3.334634,    time: 1.799954 seconds

    vlb -2.359158 (-2.

...

it 503/2000, vlb -2.799752,    time: 1.922454 seconds

    vlb -2.177633 (-1.998723)

it 504/2000, vlb -2.832146,    time: 1.935340 seconds

    vlb -2.162543 (-1.998723)

it 505/2000, vlb -2.819397,    time: 1.873298 seconds

    vlb -2.184785 (-1.998723)

it 506/2000, vlb -2.807158,    time: 1.841677 seconds

    vlb -2.232241 (-1.998723)

it 507/2000, vlb -2.804971,    time: 2.046750 seconds

    vlb -2.229083 (-1.998723)

it 508/2000, vlb -2.780258,    time: 2.034997 seconds

    vlb -2.151479 (-1.998723)

it 509/2000, vlb -2.810008,    time: 1.822101 seconds

    vlb -2.061136 (-1.998723)

it 510/2000, vlb -2.766865,    time: 1.554362 seconds

    vlb -2.088377 (-1.998723)

it 511/2000, vlb -2.821385,    time: 1.489643 seconds

    vlb -2.244368 (-1.998723)

it 512/2000, vlb -2.782950,    time: 1.789505 seconds

    vlb -2.216689 (-1.998723)

it 513/2000, vlb -2.817904,    time: 2.151731 seconds

    vlb -2.130452 (-1.998723)

it 514/2000, vlb -2.863034,    time: 1.932396 seconds


it 582/2000, vlb -2.751141,    time: 1.917230 seconds

    vlb -2.058791 (-1.998723)

it 583/2000, vlb -2.840520,    time: 1.594543 seconds

    vlb -2.124217 (-1.998723)

it 584/2000, vlb -2.781919,    time: 1.681414 seconds

    vlb -1.975160 (-1.998723)

Writting ../saves/fc_preact_VAEAC_NEW_compas_models/theta_best.dat

it 585/2000, vlb -2.767593,    time: 1.590825 seconds

    vlb -2.065042 (-1.975160)

it 586/2000, vlb -2.783789,    time: 1.649249 seconds

    vlb -2.215234 (-1.975160)

it 587/2000, vlb -2.809114,    time: 1.570287 seconds

    vlb -2.227826 (-1.975160)

it 588/2000, vlb -2.820382,    time: 1.530960 seconds

    vlb -2.063033 (-1.975160)

it 589/2000, vlb -2.768267,    time: 1.592077 seconds

    vlb -2.136804 (-1.975160)

it 590/2000, vlb -2.774396,    time: 1.521016 seconds

    vlb -2.032209 (-1.975160)

it 591/2000, vlb -2.784359,    time: 1.509726 seconds

    vlb -2.063433 (-1.975160)

it 592/2000, vlb -2.794802,    time: 1.491103 seconds

    vlb -2.141265

    vlb -2.231783 (-1.975160)

it 661/2000, vlb -2.767818,    time: 1.799035 seconds

    vlb -2.113998 (-1.975160)

it 662/2000, vlb -2.776016,    time: 1.551502 seconds

    vlb -2.187677 (-1.975160)

it 663/2000, vlb -2.733281,    time: 1.541415 seconds

    vlb -2.199460 (-1.975160)

it 664/2000, vlb -2.847155,    time: 1.566950 seconds

    vlb -2.187475 (-1.975160)

it 665/2000, vlb -2.756974,    time: 1.543407 seconds

    vlb -2.221610 (-1.975160)

it 666/2000, vlb -2.769070,    time: 1.600021 seconds

    vlb -2.110461 (-1.975160)

it 667/2000, vlb -2.762998,    time: 1.546951 seconds

    vlb -2.147192 (-1.975160)

it 668/2000, vlb -2.742754,    time: 1.547191 seconds

    vlb -2.201239 (-1.975160)

it 669/2000, vlb -2.746374,    time: 1.565629 seconds

    vlb -2.083755 (-1.975160)

it 670/2000, vlb -2.759847,    time: 1.570588 seconds

    vlb -2.141832 (-1.975160)

it 671/2000, vlb -2.763595,    time: 1.530063 seconds

    vlb -2.049889 (-1.975160)

it 672/2000, vlb -2.769

    vlb -2.178837 (-1.975160)

it 740/2000, vlb -2.754018,    time: 1.706073 seconds

    vlb -2.104683 (-1.975160)

it 741/2000, vlb -2.793467,    time: 1.831236 seconds

    vlb -2.136558 (-1.975160)

it 742/2000, vlb -2.724116,    time: 1.832630 seconds

    vlb -2.155823 (-1.975160)

it 743/2000, vlb -2.776677,    time: 1.577784 seconds

    vlb -2.142561 (-1.975160)

it 744/2000, vlb -2.731278,    time: 1.511984 seconds

    vlb -2.179783 (-1.975160)

it 745/2000, vlb -2.766428,    time: 1.553958 seconds

    vlb -2.181906 (-1.975160)

it 746/2000, vlb -2.695700,    time: 1.628538 seconds

    vlb -2.263817 (-1.975160)

it 747/2000, vlb -2.765636,    time: 1.544713 seconds

    vlb -2.196851 (-1.975160)

it 748/2000, vlb -2.733151,    time: 1.540051 seconds

    vlb -2.204757 (-1.975160)

it 749/2000, vlb -2.767974,    time: 1.575416 seconds

    vlb -2.099112 (-1.975160)

it 750/2000, vlb -2.783411,    time: 1.491974 seconds

    vlb -2.172419 (-1.975160)

it 751/2000, vlb -2.765

NameError: name 'humansize' is not defined

## Credit

In [19]:
masker = top_masker(p=1)

x_train, x_test, x_means, x_stds = \
load_UCI(dset_name='default_credit', splits=10, seed=42, separate_targets=False, save_dir='../data/')

input_dim_vec = [1, 2, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1 ,1 ,1 ,1 ,1, 2] #has this form for targets
print(input_dim_vec)

x_train = unnormalise_cat_vars(x_train, x_means, x_stds, input_dim_vec)
x_test = unnormalise_cat_vars(x_test, x_means, x_stds, input_dim_vec)

print(x_train.shape)
print(x_test.shape)

dname = 'default_credit'
print(dname)

trainset = Datafeed(x_train, x_train, transform=None)
valset = Datafeed(x_test, x_test, transform=None)

save_dir = '../saves/fc_preact_VAEAC_NEW2_' + dname

width = widths[names.index(dname)]
depth = depths[names.index(dname)] # number of hidden layers
latent_dim = latent_dims[names.index(dname)]

batch_size = 64
nb_epochs = 2000
early_stop = 200
lr = 7e-4

cuda = torch.cuda.is_available()

net = VAEAC_gauss_cat_net(input_dim_vec, width, depth, latent_dim, pred_sig=False, lr=lr, cuda=cuda, flatten=True)


vlb_train, vlb_dev = train_VAEAC(net, masker, save_dir, batch_size, nb_epochs, trainset, valset, cuda,
                     flat_ims=False, train_plot=False, Nclass=None, early_stop=early_stop)





[1, 2, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]
(27000, 24)
(3000, 24)
default_credit

Net:
VAE_gauss_net
    Total params: 0.80M

Network:

Train:
  init cost variables:


/var/folders/38/m6rs_5bd7zsf3gjy2hg8y57r0000gn/T/ipykernel_31096/1070947545.py:6: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(p.numel() for p in self.model.parameters())


it 0/2000, vlb -19.182486,    time: 16.533756 seconds

    vlb -9.254449 (-inf)

Writting ../saves/fc_preact_VAEAC_NEW2_default_credit_models/theta_best.dat

it 1/2000, vlb -11.698959,    time: 18.969596 seconds

    vlb -7.840795 (-9.254449)

Writting ../saves/fc_preact_VAEAC_NEW2_default_credit_models/theta_best.dat

it 2/2000, vlb -10.394569,    time: 15.788894 seconds

    vlb -7.176111 (-7.840795)

Writting ../saves/fc_preact_VAEAC_NEW2_default_credit_models/theta_best.dat

it 3/2000, vlb -9.829430,    time: 17.466390 seconds

    vlb -6.881644 (-7.176111)

Writting ../saves/fc_preact_VAEAC_NEW2_default_credit_models/theta_best.dat

it 4/2000, vlb -9.581752,    time: 16.366702 seconds

    vlb -6.702829 (-6.881644)

Writting ../saves/fc_preact_VAEAC_NEW2_default_credit_models/theta_best.dat



KeyboardInterrupt: 

In [ ]:
print(net.model)

## Under VAEAC

In [19]:
class VAE_gauss(nn.Module):
    def __init__(self, input_dim, width, depth, latent_dim, pred_sig=True):
        super(VAE_gauss, self).__init__()

        self.encoder = MLP_preact_recognition_net(input_dim, width, depth, latent_dim)
        if pred_sig:
            self.decoder = MLP_preact_generator_net(2*input_dim, width, depth, latent_dim)
            self.rec_loglike = GaussianLoglike(min_sigma=1e-2)
        else:
            self.decoder = MLP_preact_generator_net(input_dim, width, depth, latent_dim)
            self.m_rec_loglike = MSELoss(reduction='none')
        self.pred_sig = pred_sig

    def encode(self, x):
        approx_post_params = self.encoder(x)
        approx_post = normal_parse_params(approx_post_params, 1e-3)
        return approx_post

    def decode(self, z_sample):
        rec_params = self.decoder(z_sample)
        return rec_params

    def vlb(self, prior, approx_post, x, rec_params):
        if self.pred_sig:
            rec = self.rec_loglike(rec_params, x).view(x.shape[0], -1).sum(-1)
        else:
            rec = -self.m_rec_loglike(rec_params, x).view(x.shape[0], -1).sum(-1)
        kl = kl_divergence(approx_post, prior).view(x.shape[0], -1).sum(-1)
        return rec - kl

    def iwlb(self, prior, approx_post, x, K=50):
        estimates = []
        for i in range(K):
            latent = approx_post.rsample()
            rec_params = self.decode(latent)
            if self.pred_sig:
                rec_loglike = self.rec_loglike(rec_params, x).view(x.shape[0], -1).sum(-1)
            else:
                rec_loglike = -self.m_rec_loglike(rec_params, x).view(x.shape[0], -1).sum(-1)

            prior_log_prob = prior.log_prob(latent)
            prior_log_prob = prior_log_prob.view(x.shape[0], -1)
            prior_log_prob = prior_log_prob.sum(-1)

            proposal_log_prob = approx_post.log_prob(latent)
            proposal_log_prob = proposal_log_prob.view(x.shape[0], -1)
            proposal_log_prob = proposal_log_prob.sum(-1)

            estimate = rec_loglike + prior_log_prob - proposal_log_prob
            estimates.append(estimate[:, None])

        return torch.logsumexp(torch.cat(estimates, 1), 1) - np.log(K)


class VAE_gauss_net(BaseNet):
    def __init__(self, input_dim, width, depth, latent_dim, pred_sig=True, lr=1e-3, cuda=True):
        super(VAE_gauss_net, self).__init__()
        cprint('y', 'VAE_gauss_net')

        self.cuda = cuda

        self.input_dim = input_dim
        self.width = width
        self.depth = depth
        self.latent_dim = latent_dim
        self.lr = lr
        self.pred_sig = pred_sig

        self.create_net()
        self.create_opt()
        self.epoch = 0
        self.schedule = None

        if self.cuda:
            self.prior = self.prior = Normal(loc=torch.zeros(latent_dim).cuda(), scale=torch.ones(latent_dim).cuda())
        else:
            self.prior = Normal(loc=torch.zeros(latent_dim), scale=torch.ones(latent_dim))
        self.vlb_scale = 1 / input_dim  # scale for dimensions of input so we can use same LR always

    def create_net(self):
        torch.manual_seed(42)
        torch.cuda.manual_seed(42)
        self.model = VAE_gauss(self.input_dim, self.width, self.depth, self.latent_dim, self.pred_sig)
        if self.cuda:
            self.model = self.model.cuda()
            cudnn.benchmark = True
        print('    Total params: %.2fM' % (self.get_nb_parameters() / 1000000.0))

    def create_opt(self):
        self.optimizer = RAdam(self.model.parameters(), lr=self.lr)

    def fit(self, x):
        self.set_mode_train(train=True)

        x, = to_variable(var=(x, ), cuda=self.cuda)
        self.optimizer.zero_grad()

        approx_post = self.model.encode(x)
        z_sample = approx_post.rsample()
        rec_params = self.model.decode(z_sample)

        vlb = self.model.vlb(self.prior, approx_post, x, rec_params)
        loss = (- vlb * self.vlb_scale).mean()

        loss.backward()
        self.optimizer.step()

        return vlb.mean().item(), rec_params

    def eval(self, x, sample=False):
        self.set_mode_train(train=False)

        x, = to_variable(var=(x, ), cuda=self.cuda)
        approx_post = self.model.encode(x)
        if sample:
            z_sample = approx_post.sample()
        else:
            z_sample = approx_post.loc
        rec_params = self.model.decode(z_sample)

        vlb = self.model.vlb(self.prior, approx_post, x, rec_params)

        return vlb.mean().item(), rec_params

    def eval_iw(self, x, k=50):
        self.set_mode_train(train=False)
        x,  = to_variable(var=(x, ), cuda=self.cuda)

        approx_post = self.model.recognition_encode(x)

        iw_lb = self.model.iwlb(self.prior, approx_post, x, k)
        return iw_lb.mean().item()

    def recongnition(self, x, grad=False):
        self.set_mode_train(train=False)
        if grad:
            if not x.requires_grad:
                x.requires_grad = True
        else:
            x, = to_variable(var=(x,), volatile=True, cuda=self.cuda)
        approx_post = self.model.encode(x)
        return approx_post

    def regenerate(self, z, grad=False):
        self.set_mode_train(train=False)
        if grad:
            if not z.requires_grad:
                z.requires_grad = True
        else:
            z, = to_variable(var=(z,), volatile=True, cuda=self.cuda)
        out = self.model.decode(z)
        if self.pred_sig:
            return normal_parse_params(out, 1e-2)
        else:
            return out

In [20]:
def train_VAE(net, name, batch_size, nb_epochs, trainset, valset, cuda, flat_ims=False,
              train_plot=False, Nclass=None, early_stop=None, script_mode=False):

    models_dir = name + '_models'
    results_dir = name + '_results'
    mkdir(models_dir)
    mkdir(results_dir)

    if cuda:
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True,
                                                  num_workers=0)
        valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=True,
                                                num_workers=0)

    else:
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=False,
                                                  num_workers=0)
        valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=False,
                                                num_workers=0)

## ---------------------------------------------------------------------------------------------------------------------
# net dims
    cprint('c', '\nNetwork:')

    epoch = 0

    ## ---------------------------------------------------------------------------------------------------------------------
    # train
    cprint('c', '\nTrain:')

    print('  init cost variables:')
    vlb_train = np.zeros(nb_epochs)
    vlb_dev = np.zeros(nb_epochs)
    best_vlb = -np.inf
    best_vlb_train = -np.inf
    best_epoch = 0

    nb_its_dev = 1

    tic0 = time.time()
    for i in range(epoch, nb_epochs):
        net.set_mode_train(True)

        tic = time.time()
        nb_samples = 0
        for x, y in trainloader:

            if flat_ims:
                x = x.view(x.shape[0], -1)
            if Nclass is not None:
                y_oh = torch_onehot(y, Nclass).type(x.type())
                x = torch.cat([x, y_oh], 1)

            cost, _ = net.fit(x)

            vlb_train[i] += cost * len(x)
            nb_samples += len(x)

        vlb_train[i] /= nb_samples

        toc = time.time()

        # ---- print
        print("it %d/%d, vlb %f, " % (i, nb_epochs, vlb_train[i]), end="")
        cprint('r', '   time: %f seconds\n' % (toc - tic))
        net.update_lr(i)

        if vlb_train[i] > best_vlb_train:
            best_vlb_train = vlb_train[i]

        # ---- dev
        if i % nb_its_dev == 0:
            nb_samples = 0
            for j, (x, y) in enumerate(valloader):

                if flat_ims:
                    x = x.view(x.shape[0], -1)
                if Nclass is not None:
                    y_oh = torch_onehot(y, Nclass).type(x.type())
                    x = torch.cat([x, y_oh], 1)

                cost, _ = net.eval(x)

                vlb_dev[i] += cost * len(x)
                nb_samples += len(x)

            vlb_dev[i] /= nb_samples

            cprint('g', '    vlb %f (%f)\n' % (vlb_dev[i], best_vlb))

            if train_plot:
                zz = net.recongnition(x).sample()
                o = net.regenerate(zz)
                try:
                    o = o.cpu()
                except:
                    o = o.loc.cpu()
                if len(x.shape) == 2:
                    side = int(np.sqrt(x.shape[1]))
                    x = x.view(-1, 1, side, side).data
                    o = o.view(-1, 1, side, side).data

                # save_image(torch.cat([x[:8], o[:8]]), results_dir + '/rec_%d.png' % i, nrow=8)
                import matplotlib.pyplot as plt
                plt.figure()
                dd = make_grid(torch.cat([x[:10], o[:10]]), nrow=10).numpy()
                plt.imshow(np.transpose(dd, (1, 2, 0)), interpolation='nearest')
                if script_mode:
                    plt.savefig(results_dir + '/rec%d.png' % i)
                else:
                    plt.show()

                z_sample = normal(loc=0.0, scale=1.0, size=(36, net.latent_dim))
                x_rec = net.regenerate(z_sample)
                try:
                    x_rec = x_rec.cpu()
                except:
                    x_rec = x_rec.loc.cpu()
                if len(x_rec.shape) == 2:
                    side = int(np.sqrt(x_rec.shape[1]))
                    x_rec = x_rec.view(-1, 1, side, side)
                plt.figure()
                dd = make_grid(x_rec, nrow=6).numpy()
                plt.imshow(np.transpose(dd, (1, 2, 0)), interpolation='nearest')
                if script_mode:
                    plt.savefig(results_dir + '/sample%d.png' % i)
                else:
                    plt.show()

        if vlb_dev[i] > best_vlb:
            best_vlb = vlb_dev[i]
            best_epoch = i
            net.save(models_dir + '/theta_best.dat')

        if early_stop is not None and (i - best_epoch) > early_stop:
            break


    net.save(models_dir + '/theta_last.dat')
    toc0 = time.time()
    runtime_per_it = (toc0 - tic0) / float(nb_epochs)
    cprint('r', '   average time: %f seconds\n' % runtime_per_it)

    ## ---------------------------------------------------------------------------------------------------------------------
    # results
    cprint('c', '\nRESULTS:')
    nb_parameters = net.get_nb_parameters()
    best_cost_dev = best_vlb
    best_cost_train = best_vlb_train

    print('  best_vlb_dev: %f' % best_cost_dev)
    print('  best_vlb_train: %f' % best_cost_train)
    print('  nb_parameters: %d (%s)\n' % (nb_parameters, humansize(nb_parameters)))

    ## ---------------------------------------------------------------------------------------------------------------------
    # fig cost vs its
    if not train_plot:
        import matplotlib
        matplotlib.use('agg')
    import matplotlib.pyplot as plt
    if train_plot:
        plt.figure()
        plt.plot(np.clip(vlb_train, -1000, 1000), 'r')
        plt.plot(np.clip(vlb_dev[::nb_its_dev], -1000, 1000), 'b')
        plt.legend(['cost_train', 'cost_dev'])
        plt.ylabel('vlb')
        plt.xlabel('it')
        plt.grid(True)
        plt.savefig(results_dir+'/train_cost.png')
        if train_plot:
            plt.show()
    return vlb_train, vlb_dev





In [21]:
class under_VAEAC(BaseNet):
    def __init__(self, base_VAE, width, depth, latent_dim, lr=1e-3, cuda=True):
        super(under_VAEAC, self).__init__()
        cprint('y', 'VAE_gauss_net')

        self.base_VAEAC = base_VAE
        self.pred_sig = False

        self.cuda = cuda

        self.input_dim = self.base_VAEAC.latent_dim
        self.width = width
        self.depth = depth
        self.latent_dim = latent_dim
        self.lr = lr

        self.create_net()
        self.create_opt()
        self.epoch = 0
        self.schedule = None

        if self.cuda:
            self.prior = self.prior = Normal(loc=torch.zeros(latent_dim).cuda(), scale=torch.ones(latent_dim).cuda())
        else:
            self.prior = Normal(loc=torch.zeros(latent_dim), scale=torch.ones(latent_dim))
        self.vlb_scale = 1 / self.input_dim  # scale for dimensions of input so we can use same LR always

    def create_net(self):
        torch.manual_seed(42)
        torch.cuda.manual_seed(42)
        self.model = VAE_gauss(self.input_dim, self.width, self.depth, self.latent_dim, self.pred_sig)
        if self.cuda:
            self.model = self.model.cuda()
            cudnn.benchmark = True
        print('    Total params: %.2fM' % (self.get_nb_parameters() / 1000000.0))

    def create_opt(self):
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)

    def fit(self, x):
        self.set_mode_train(train=True)

        x, = to_variable(var=(x, ), cuda=self.cuda)
        self.optimizer.zero_grad()

        z_sample = self.base_VAEAC.recognition_encode(x).sample()
        approx_post = self.model.encode(z_sample)
        u_sample = approx_post.rsample()
        rec_params = self.model.decode(u_sample)

        vlb = self.model.vlb(self.prior, approx_post, z_sample, rec_params)
        loss = (- vlb * self.vlb_scale).mean()

        loss.backward()
        self.optimizer.step()

        return vlb.mean().item(), rec_params

    def eval(self, x):

        self.set_mode_train(train=False)

        x, = to_variable(var=(x, ), cuda=self.cuda)
        z_sample = self.base_VAEAC.recognition_encode(x).sample()
        approx_post = self.model.encode(z_sample)
        u_sample = approx_post.rsample()
        rec_params = self.model.decode(u_sample)

        vlb = self.model.vlb(self.prior, approx_post, z_sample, rec_params)

        return vlb.mean().item(), rec_params

    def eval_iw(self, x, k=50):
        self.set_mode_train(train=False)
        x,  = to_variable(var=(x, ), cuda=self.cuda)
        z_sample = self.base_VAEAC.recognition_encode(x).sample()
        approx_post = self.model.recognition_encode(z_sample)

        iw_lb = self.model.iwlb(self.prior, approx_post, z_sample, k)
        return iw_lb.mean().item()

    def recongnition(self, x, grad=False):
        self.set_mode_train(train=False)
        if grad:
            if not x.requires_grad:
                x.requires_grad = True
        else:
            x, = to_variable(var=(x,), volatile=True, cuda=self.cuda)
        approx_post = self.model.encode(x)
        return approx_post

    def regenerate(self, z, grad=False):
        self.set_mode_train(train=False)
        if grad:
            if not z.requires_grad:
                z.requires_grad = True
        else:
            z, = to_variable(var=(z,), volatile=True, cuda=self.cuda)
        out = self.model.decode(z)
        return out.data

    def u_recongnition(self, x, grad=False):
        self.set_mode_train(train=False)
        if grad:
            if not x.requires_grad:
                x.requires_grad = True
        else:
            x, = to_variable(var=(x,), volatile=True, cuda=self.cuda)

        z = self.base_VAEAC.recognition_encode(x).loc
        approx_post = self.model.encode(z)
        return approx_post

    def u_mask_recongnition(self, x, mask, grad=False):
        self.set_mode_train(train=False)
        if grad:
            if not x.requires_grad:
                x.requires_grad = True
            mask, = to_variable(var=(mask, ), cuda=self.cuda)
        else:
            x, mask = to_variable(var=(x, mask), cuda=self.cuda)

        z = self.base_VAEAC.prior_encode(x, mask).loc
        approx_post = self.model.encode(z)
        return approx_post

    def u_regenerate(self, u, grad=False):
        self.set_mode_train(train=False)
        if grad:
            if not u.requires_grad:
                u.requires_grad = True
        else:
            u, = to_variable(var=(u,), volatile=True, cuda=self.cuda)

        z = self.model.decode(u)
        out = self.base_VAEAC.decode(z)
        if self.base_VAEAC.pred_sig:
            return normal_parse_params(out, 1e-2)
        else:
            return out.data

In [27]:
# Credit
"""
dname = 'default_credit'
print(dname)

x_train, x_test, x_means, x_stds = \
load_UCI(dset_name=dname, splits=10, seed=42, separate_targets=False, save_dir='../data/')

input_dim_vec = [1, 2, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1 ,1 ,1 ,1 ,1, 2] #has this form for targets
print(input_dim_vec)

x_train = unnormalise_cat_vars(x_train, x_means, x_stds, input_dim_vec)
x_test = unnormalise_cat_vars(x_test, x_means, x_stds, input_dim_vec)

x_train_flat = gauss_cat_to_flat(torch.Tensor(x_train), input_dim_vec)
x_test_flat = gauss_cat_to_flat(torch.Tensor(x_test), input_dim_vec)


print(x_train_flat.shape)
print(x_test_flat.shape)

trainset = Datafeed(x_train_flat, x_train_flat, transform=None)
valset = Datafeed(x_test_flat, x_test_flat, transform=None)


width = widths[names.index(dname)]
depth = depths[names.index(dname)] # number of hidden layers
latent_dim = latent_dims[names.index(dname)]
lr = 1e-4

cuda = torch.cuda.is_available()

#base_network_wraper = VAEAC_gauss_cat_net(input_dim_vec, width, depth, latent_dim, pred_sig=False, lr=lr, cuda=cuda, flatten=True)
#base_network_wraper.load('../saves/fc_preact_VAEAC_NEW2_' + dname + '_models/theta_best.dat')

base_network = net.model
width = 150
depth = 2
latent_dim = under_latent_dims2[names.index(dname)]

batch_size = 128
nb_epochs = 2000
early_stop = 200
lr = 1e-4

"""

# COMPAS
#"""
x_train, x_test, x_means, x_stds, y_train, y_test, feature_names, X_dims = \
    get_my_COMPAS(rseed=42, separate_test=True, test_ratio=0.1, save_dir='../data/')

x_train, x_test, input_dim_vec = join_compas_targets(x_train, x_test, y_train, y_test, X_dims)
print('Compas', x_train.shape, x_test.shape)
print(input_dim_vec)

dname = 'compas'
print(dname)

trainset = Datafeed(x_train, x_train, transform=None)
valset = Datafeed(x_test, x_test, transform=None)

width = 150
depth = 2
latent_dim = under_latent_dims[names.index(dname)]

batch_size = 128
nb_epochs = 2000
early_stop = 200
lr = 1e-4

cuda = False

base_network_wraper = VAEAC_gauss_cat_net(input_dim_vec, width, depth, latent_dim, pred_sig=False, lr=lr, cuda=cuda, flatten=True)
base_network_wraper.load('../saves/fc_preact_VAEAC_NEW_' + dname + '_models/theta_best.dat')

#VAEAC = VAEAC_gauss_cat_net(input_dim_vec, width, depth, latent_dim, pred_sig=False, lr=lr, cuda=cuda, flatten=flat_vaeac_bools[d_idx])
#VAEAC.load('../saves/fc_preact_VAEAC_NEW_' + dname + '_models/theta_best.dat')

base_network = base_network_wraper.model

savedir = '../saves/fc_VAEAC_NEW_under2_' + dname  # remove the 2 to remove dimensionality reduction

cuda = torch.cuda.is_available()

under_VAEAC_net = under_VAEAC(base_network, width, depth, latent_dim, lr, cuda=cuda)

vlb_train, vlb_dev = train_VAE(under_VAEAC_net, savedir,
                               batch_size, nb_epochs, trainset, valset, cuda=cuda,
                               flat_ims=False, train_plot=False, Nclass=None, early_stop=early_stop)


Compas (5554, 19) (618, 19)
[3 6 2 2 2 1 1 2]
compas

Net:
VAE_gauss_net
    Total params: 0.08M
Reading ../saves/fc_preact_VAEAC_NEW_compas_models/theta_best.dat

  restoring epoch: 585, lr: 0.000100

Net:
VAE_gauss_net
    Total params: 0.05M

Network:

Train:
  init cost variables:


/var/folders/38/m6rs_5bd7zsf3gjy2hg8y57r0000gn/T/ipykernel_17901/1070947545.py:6: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(p.numel() for p in self.model.parameters())


it 0/2000, vlb -14.701037,    time: 0.784924 seconds

    vlb -11.398861 (-inf)

Writting ../saves/fc_VAEAC_NEW_under2_compas_models/theta_best.dat

it 1/2000, vlb -10.097281,    time: 0.645048 seconds

    vlb -9.089679 (-11.398861)

Writting ../saves/fc_VAEAC_NEW_under2_compas_models/theta_best.dat

it 2/2000, vlb -8.366908,    time: 0.632210 seconds

    vlb -7.923545 (-9.089679)

Writting ../saves/fc_VAEAC_NEW_under2_compas_models/theta_best.dat

it 3/2000, vlb -7.598868,    time: 0.562514 seconds

    vlb -7.292997 (-7.923545)

Writting ../saves/fc_VAEAC_NEW_under2_compas_models/theta_best.dat

it 4/2000, vlb -7.387952,    time: 0.750483 seconds

    vlb -7.290502 (-7.292997)

Writting ../saves/fc_VAEAC_NEW_under2_compas_models/theta_best.dat

it 5/2000, vlb -7.300517,    time: 0.669122 seconds

    vlb -6.916430 (-7.290502)

Writting ../saves/fc_VAEAC_NEW_under2_compas_models/theta_best.dat

it 6/2000, vlb -7.153251,    time: 0.910711 seconds

    vlb -7.047253 (-6.916430)

it 7/

    vlb -5.944459 (-5.918562)

it 66/2000, vlb -6.146011,    time: 0.778199 seconds

    vlb -5.880102 (-5.918562)

Writting ../saves/fc_VAEAC_NEW_under2_compas_models/theta_best.dat

it 67/2000, vlb -6.141405,    time: 0.663560 seconds

    vlb -6.003874 (-5.880102)

it 68/2000, vlb -6.139710,    time: 0.598890 seconds

    vlb -5.981514 (-5.880102)

it 69/2000, vlb -6.080537,    time: 0.622848 seconds

    vlb -5.951593 (-5.880102)

it 70/2000, vlb -6.146163,    time: 0.627627 seconds

    vlb -5.947422 (-5.880102)

it 71/2000, vlb -6.105737,    time: 0.572939 seconds

    vlb -5.899740 (-5.880102)

it 72/2000, vlb -6.109804,    time: 0.527234 seconds

    vlb -6.096789 (-5.880102)

it 73/2000, vlb -6.130235,    time: 0.537746 seconds

    vlb -5.973827 (-5.880102)

it 74/2000, vlb -6.104355,    time: 0.563269 seconds

    vlb -6.019366 (-5.880102)

it 75/2000, vlb -6.154775,    time: 1.172286 seconds

    vlb -5.889835 (-5.880102)

it 76/2000, vlb -6.127846,    time: 1.692851 second

it 142/2000, vlb -6.118712,    time: 0.695020 seconds

    vlb -6.024364 (-5.755878)

it 143/2000, vlb -6.071700,    time: 0.558199 seconds

    vlb -5.955481 (-5.755878)

it 144/2000, vlb -6.025098,    time: 0.549696 seconds

    vlb -5.931567 (-5.755878)

it 145/2000, vlb -6.072117,    time: 0.557927 seconds

    vlb -5.921461 (-5.755878)

it 146/2000, vlb -6.040099,    time: 0.561406 seconds

    vlb -5.900092 (-5.755878)

it 147/2000, vlb -6.094528,    time: 0.547864 seconds

    vlb -5.864831 (-5.755878)

it 148/2000, vlb -6.048404,    time: 0.779322 seconds

    vlb -5.979649 (-5.755878)

it 149/2000, vlb -6.108491,    time: 1.004430 seconds

    vlb -5.919129 (-5.755878)

it 150/2000, vlb -6.056390,    time: 0.733275 seconds

    vlb -5.900243 (-5.755878)

it 151/2000, vlb -6.036307,    time: 0.634225 seconds

    vlb -5.928283 (-5.755878)

it 152/2000, vlb -6.068594,    time: 0.951651 seconds

    vlb -5.881363 (-5.755878)

it 153/2000, vlb -6.053750,    time: 1.286238 seconds


    vlb -5.985422 (-5.752071)

it 221/2000, vlb -6.032331,    time: 0.641286 seconds

    vlb -6.032875 (-5.752071)

it 222/2000, vlb -6.062815,    time: 0.615746 seconds

    vlb -5.801141 (-5.752071)

it 223/2000, vlb -6.058413,    time: 0.619095 seconds

    vlb -5.948525 (-5.752071)

it 224/2000, vlb -6.024577,    time: 0.637255 seconds

    vlb -5.980161 (-5.752071)

it 225/2000, vlb -6.031155,    time: 0.618182 seconds

    vlb -5.837657 (-5.752071)

it 226/2000, vlb -6.034582,    time: 0.701375 seconds

    vlb -5.920137 (-5.752071)

it 227/2000, vlb -6.010272,    time: 0.818609 seconds

    vlb -6.050950 (-5.752071)

it 228/2000, vlb -6.042468,    time: 0.611920 seconds

    vlb -5.997070 (-5.752071)

it 229/2000, vlb -5.969879,    time: 0.609394 seconds

    vlb -5.920860 (-5.752071)

it 230/2000, vlb -6.032652,    time: 0.569449 seconds

    vlb -5.967976 (-5.752071)

it 231/2000, vlb -6.036839,    time: 0.570477 seconds

    vlb -5.747014 (-5.752071)

Writting ../saves/fc_VA

it 298/2000, vlb -5.970429,    time: 0.570001 seconds

    vlb -5.915905 (-5.722203)

it 299/2000, vlb -6.031841,    time: 0.569940 seconds

    vlb -5.916170 (-5.722203)

it 300/2000, vlb -6.065870,    time: 0.570196 seconds

    vlb -5.850285 (-5.722203)

it 301/2000, vlb -6.016728,    time: 0.571005 seconds

    vlb -5.789905 (-5.722203)

it 302/2000, vlb -6.038583,    time: 0.574775 seconds

    vlb -5.873112 (-5.722203)

it 303/2000, vlb -6.046880,    time: 0.573638 seconds

    vlb -5.832935 (-5.722203)

it 304/2000, vlb -5.981013,    time: 0.580135 seconds

    vlb -5.941791 (-5.722203)

it 305/2000, vlb -6.037195,    time: 0.571474 seconds

    vlb -5.765086 (-5.722203)

it 306/2000, vlb -5.954249,    time: 0.568071 seconds

    vlb -5.985123 (-5.722203)

it 307/2000, vlb -6.039229,    time: 0.588812 seconds

    vlb -5.785161 (-5.722203)

it 308/2000, vlb -6.005060,    time: 0.569245 seconds

    vlb -5.833501 (-5.722203)

it 309/2000, vlb -5.978019,    time: 0.575960 seconds


it 377/2000, vlb -6.032511,    time: 0.555007 seconds

    vlb -5.908240 (-5.722203)

it 378/2000, vlb -5.972814,    time: 0.583343 seconds

    vlb -5.874425 (-5.722203)

it 379/2000, vlb -5.964904,    time: 0.538458 seconds

    vlb -5.829979 (-5.722203)

it 380/2000, vlb -6.021003,    time: 0.531327 seconds

    vlb -5.877202 (-5.722203)

it 381/2000, vlb -6.001778,    time: 0.587768 seconds

    vlb -5.858331 (-5.722203)

it 382/2000, vlb -6.032161,    time: 0.721274 seconds

    vlb -5.791344 (-5.722203)

it 383/2000, vlb -6.031584,    time: 0.709136 seconds

    vlb -5.892537 (-5.722203)

it 384/2000, vlb -5.990223,    time: 0.594025 seconds

    vlb -5.757034 (-5.722203)

it 385/2000, vlb -6.018463,    time: 0.663535 seconds

    vlb -5.824331 (-5.722203)

it 386/2000, vlb -5.994739,    time: 0.686636 seconds

    vlb -5.852992 (-5.722203)

it 387/2000, vlb -5.975746,    time: 0.639118 seconds

    vlb -5.927864 (-5.722203)

it 388/2000, vlb -6.034774,    time: 0.676868 seconds


NameError: name 'humansize' is not defined

In [29]:
under_VAEAC_net.get_nb_parameters()

/var/folders/38/m6rs_5bd7zsf3gjy2hg8y57r0000gn/T/ipykernel_10249/1070947545.py:6: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(p.numel() for p in self.model.parameters())


49812

In [50]:
number_p = 0
for p in net.model.parameters():
    for i in range(len(p.data)):
        #print(len(p.data))
        try:
            number_p += len(p.data[i])
        except:
            number_p += 1
print(number_p)

778085


In [51]:
for p in net.model.parameters():
    print(p.data)

tensor([[ 0.1953,  0.1680, -0.0635,  ...,  0.1309,  0.0337, -0.1151],
        [ 0.0867, -0.1051, -0.0316,  ...,  0.1187,  0.0668,  0.1751],
        [ 0.0473, -0.1043,  0.0623,  ...,  0.0617,  0.0875,  0.0812],
        ...,
        [-0.1474,  0.2562, -0.1020,  ...,  0.1505,  0.1124,  0.0671],
        [-0.0777,  0.1288,  0.0823,  ...,  0.0869,  0.1883, -0.0504],
        [ 0.0840,  0.1529,  0.0006,  ...,  0.0037, -0.1448, -0.0819]])
tensor([-0.0420,  0.0951,  0.0312, -0.0081,  0.0587, -0.1466,  0.2107,  0.1487,
         0.1520, -0.1944,  0.2045,  0.0413, -0.2018,  0.2153, -0.0358,  0.0860,
         0.1789,  0.1633,  0.2059, -0.2002,  0.2350,  0.0600,  0.0048, -0.1823,
        -0.1383,  0.2303,  0.1632,  0.1677, -0.0273, -0.1063,  0.0492, -0.0949,
        -0.1412,  0.0960,  0.2115, -0.1593, -0.2120,  0.0359, -0.0733, -0.1013,
        -0.0405,  0.0021, -0.0675,  0.1887,  0.0983,  0.0666, -0.0523, -0.0020,
         0.1542,  0.0323, -0.2225,  0.0058,  0.0914, -0.0639, -0.0398, -0.0056,
      

In [53]:
for p in net.model.parameters():
    print(p.data)

tensor([[ 0.1953,  0.1680, -0.0635,  ...,  0.1309,  0.0337, -0.1151],
        [ 0.0867, -0.1051, -0.0316,  ...,  0.1187,  0.0668,  0.1751],
        [ 0.0473, -0.1043,  0.0623,  ...,  0.0617,  0.0875,  0.0812],
        ...,
        [-0.1474,  0.2562, -0.1020,  ...,  0.1505,  0.1124,  0.0671],
        [-0.0777,  0.1288,  0.0823,  ...,  0.0869,  0.1883, -0.0504],
        [ 0.0840,  0.1529,  0.0006,  ...,  0.0037, -0.1448, -0.0819]])
tensor([-0.0420,  0.0951,  0.0312, -0.0081,  0.0587, -0.1466,  0.2107,  0.1487,
         0.1520, -0.1944,  0.2045,  0.0413, -0.2018,  0.2153, -0.0358,  0.0860,
         0.1789,  0.1633,  0.2059, -0.2002,  0.2350,  0.0600,  0.0048, -0.1823,
        -0.1383,  0.2303,  0.1632,  0.1677, -0.0273, -0.1063,  0.0492, -0.0949,
        -0.1412,  0.0960,  0.2115, -0.1593, -0.2120,  0.0359, -0.0733, -0.1013,
        -0.0405,  0.0021, -0.0675,  0.1887,  0.0983,  0.0666, -0.0523, -0.0020,
         0.1542,  0.0323, -0.2225,  0.0058,  0.0914, -0.0639, -0.0398, -0.0056,
      